In [1]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 184, done.
remote: Total 184 (delta 0), reused 0 (delta 0), pack-reused 184
Receiving objects: 100% (184/184), 4.36 MiB | 3.92 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [2]:
!pip install tensorflow-gpu

     |████████████████████████████████| 377.0MB 46kB/s 


In [10]:
%cd gpt-2

/content/gpt-2


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 604kB 30.8MB/s 
     |████████████████████████████████| 51kB 19.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [21]:
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 979kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 38.8Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 941kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 74.2Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 4.04Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 50.6Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 50.0Mit/s]                                                       


In [0]:
!sed -i 's/top_k=0/top_k=40/g' src/interactive_conditional_samples.py

In [32]:
%cd src/

/content/gpt-2/src


In [0]:
def generate_text(seed_text="", length=None) -> str:
  import os
  import numpy as np
  import tensorflow as tf
  import json

  import model, sample, encoder

  nsamples = 1
  models_dir = "../models"
  batch_size = 1
  temperature = 1
  top_k = 30
  seed = None
  model_name='117M'
  model_name='345M'

  models_dir = os.path.expanduser(os.path.expandvars(models_dir))
  if batch_size is None:
      batch_size = 1
  assert nsamples % batch_size == 0

  enc = encoder.get_encoder(model_name, models_dir)
  hparams = model.default_hparams()
  with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
      hparams.override_from_dict(json.load(f))

  if length is None:
      length = hparams.n_ctx // 2
  elif length > hparams.n_ctx:
      raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

  with tf.Session(graph=tf.Graph()) as sess:
      context = tf.placeholder(tf.int32, [batch_size, None])
      np.random.seed(seed)
      tf.set_random_seed(seed)
      output = sample.sample_sequence(
          hparams=hparams, length=length,
          context=context,
          batch_size=batch_size,
          temperature=temperature, top_k=top_k
      )

      saver = tf.train.Saver()
      ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
      saver.restore(sess, ckpt)

      raw_text = seed_text
      context_tokens = enc.encode(raw_text)
      generated = 0
      for _ in range(nsamples // batch_size):
          out = sess.run(output, feed_dict={
              context: [context_tokens for _ in range(batch_size)]
          })[:, len(context_tokens):]
          for i in range(batch_size):
              generated += 1
              text = enc.decode(out[i])
      return text

In [0]:
def get_future_prediction():
  tag_seed = "children life death money career 2021 2022 2030 by the age of 50 "
  input_text = tag_seed + "In the year of 2049 I want to"
  time_markers = ["Next month I will", "In autumn it will be", "By the end of winter I will", "Next year I have to"]
  time_index = 0
  default_time_marker = "And then "

  new_text = input_text
  while len(new_text) - len(input_text) < 500:
    new_text += generate_text(new_text, length=40)
    period_position = new_text.rfind('.')
    new_text = new_text[:period_position+1]
    new_text += "\n"
    if time_index < len(time_markers):
      new_text += time_markers[time_index]  
      time_index += 1
    else:
      new_text += default_time_marker
#   return new_text[len(tag_seed):]
  return new_text

In [165]:
new_text = get_future_prediction()
print(new_text)


Next month I will be hosting another podcast, The Last Episode, on the latest news, thoughts and developments related to Bitcoin.
In autumn it will be interesting to see how many of you choose to stick with Bitcoin, whether it is for the long or the short term.
And then  you will have got a better idea about the nature of Bitcoin itself in the next newsletter to come out.
And then  I plan to continue discussing the evolution of Bitcoin in the first half of 2015.
Join and join the community and share on facebook, twitter, YouTube and other social media channels.
And then  we will have a forum for Bitcoin discussion.
I thank you for reading and I hope you will join in.
I have no intention of slowing down.
And then 


In [1]:
import telebot

In [2]:
token = '864065501:AAHvoUqncSS8t-_x7E1lnP7EvWJd3IM3mpM'
bot = telebot.TeleBot(token)